# Apify 数据集

>[Apify Dataset](https://docs.apify.com/platform/storage/dataset) 是一种可扩展的、仅追加的存储，具有顺序访问功能，专为存储结构化的网页抓取结果而设计，例如产品列表或 Google SERPs，然后将它们导出为各种格式，如 JSON、CSV 或 Excel。数据集主要用于保存 [Apify Actors](https://apify.com/store) 的结果，这些是用于各种网页抓取、爬取和数据提取用例的无服务器云程序。

本笔记本将展示如何将 Apify 数据集加载到 LangChain 中。

## 先决条件

您需要在 Apify 平台上拥有一个现有数据集。本示例将展示如何加载由 [Website Content Crawler](https://apify.com/apify/website-content-crawler) 生成的数据集。

In [ ]:
%pip install --upgrade --quiet langchain langchain-apify langchain-openai

首先，将 `ApifyDatasetLoader` 导入到你的源代码中：

In [2]:
from langchain_apify import ApifyDatasetLoader
from langchain_core.documents import Document

找到您的 [Apify API 令牌](https://console.apify.com/account/integrations) 和 [OpenAI API 密钥](https://platform.openai.com/account/api-keys)，并将它们初始化到环境变量中：

In [7]:
import os

os.environ["APIFY_API_TOKEN"] = "your-apify-api-token"
os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

然后提供一个函数，将 Apify 数据集记录字段映射到 LangChain `Document` 格式。

例如，如果你的数据集项结构如下：

```json
{
    "url": "https://apify.com",
    "text": "Apify 是最佳的网页抓取和自动化平台。"
}
```

下面的代码中的映射函数会将它们转换为 LangChain `Document` 格式，以便你可以将它们进一步用于任何 LLM 模型（例如用于问答）。

In [8]:
loader = ApifyDatasetLoader(
    dataset_id="your-dataset-id",
    dataset_mapping_function=lambda dataset_item: Document(
        page_content=dataset_item["text"], metadata={"source": dataset_item["url"]}
    ),
)

In [9]:
data = loader.load()

## 一个带有问答的示例

在这个示例中，我们使用数据集中的数据来回答一个问题。

In [14]:
from langchain.indexes import VectorstoreIndexCreator
from langchain_apify import ApifyWrapper
from langchain_core.documents import Document
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

In [15]:
loader = ApifyDatasetLoader(
    dataset_id="your-dataset-id",
    dataset_mapping_function=lambda item: Document(
        page_content=item["text"] or "", metadata={"source": item["url"]}
    ),
)

In [16]:
index = VectorstoreIndexCreator(
    vectorstore_cls=InMemoryVectorStore, embedding=OpenAIEmbeddings()
).from_loaders([loader])

In [17]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [23]:
query = "What is Apify?"
result = index.query_with_sources(query, llm=llm)

In [ ]:
print(result["answer"])
print(result["sources"])

 Apify is a platform for developing, running, and sharing serverless cloud programs. It enables users to create web scraping and automation tools and publish them on the Apify platform.

https://docs.apify.com/platform/actors, https://docs.apify.com/platform/actors/running/actors-in-store, https://docs.apify.com/platform/security, https://docs.apify.com/platform/actors/examples
